In [ ]:
import numpy as np
import os
import pandas as pd 
import cv2
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Dropout, Input, Conv2DTranspose, Cropping2D
import keras.utils.all_utils as kr_utils
import keras.regularizers
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [ ]:

np.random.seed = 24
imgdir = '../input/img-data/images/images'
maskdir = '../input/img-data/masks/masks'
imgs = []
masks = []

for filename in os.listdir(imgdir):
    imgpath = os.path.join(imgdir, filename)
    maskpath = os.path.join(maskdir, filename)
    imginst = cv2.imread(imgpath)
    maskinst = cv2.imread(maskpath)
    
    imginst = cv2.cvtColor(imginst, cv2.COLOR_BGR2RGB)
    imgind = np.nonzero(maskinst > 0)
    maskinst[imgind[0],imgind[1],:] = 255
    
    imgs.append(imginst)
    masks.append(maskinst)

masks = np.array(masks)
masks = masks[:,:,:,0]
masks = np.expand_dims(masks,3)
#print(np.unique(masks))
#print(masks.shape)
imgs = np.array(imgs)
plt.imshow(masks[6])
plt.plot()
X_train, X_test, y_train, y_test = train_test_split(imgs,masks,test_size = 1/10, random_state = 24)    

In [ ]:
drop_ratio = 8/100
max_conv_maps = 256

def encoders(mod_input, conv_maps, filter_sz, padding_type):
    ei = Conv2D(conv_maps,filter_sz,padding = padding_type, activation='relu')(mod_input)
    ei = Dropout(drop_ratio)(ei)
    ei = Conv2D(conv_maps,filter_sz,padding = padding_type, activation='relu')(ei)
    return ei

def decoders(enc_input, up_input, up_maps, up_sz, up_stride, padding_type, conv_maps, filter_sz, Print = False):
    up_do = Conv2DTranspose(up_maps, up_sz, strides=up_stride, padding = padding_type)(up_input)
    if (Print):
        print(up_do.shape)
    up_do = Concatenate()([enc_input,up_do])
    up_do = Conv2D(conv_maps,filter_sz,padding = padding_type, activation='relu')(up_do)
    up_do = Conv2D(conv_maps,filter_sz,padding = padding_type, activation='relu')(up_do)
    return up_do

cnn_input = Input(shape=(360, 480, 3))
eo1 = encoders(cnn_input, max_conv_maps / (2 ** 4), (3,3), 'same')
#crp1 = Cropping2D(((4,4),(0,0)))(eo1)
#print(eo1.shape)
#print(crp1.shape)
ei2 = MaxPooling2D((2,2))(eo1)

eo2 = encoders(ei2, max_conv_maps / (2 ** 3), (3,3), 'same')
#crp2 = Cropping2D(((2,2),(0,0)))(eo2)
#print(eo2.shape)
#print(crp2.shape)
ei3 = MaxPooling2D((2,2))(eo2)

eo3 = encoders(ei3, max_conv_maps / (2 ** 2), (3,3), 'same')
#crp3 = Cropping2D(((1,1),(0,0)))(eo3)
#print(eo3.shape)
#print(crp3.shape)
ei4 = MaxPooling2D((2,2))(eo3)

eo4 = encoders(ei4, max_conv_maps / (2 ** 1), (3,3), 'same')
#crp4 = Cropping2D(((1,0),(0,0)))(eo4)
#print(eo4.shape)
#print(crp4.shape)
ei5 = MaxPooling2D((3,3))(eo4)

peak = Conv2D(max_conv_maps / (2 ** 0),(3,3),padding = 'same', activation='relu')(ei5)
peak = Dropout(drop_ratio)(peak)
peak = Conv2D(max_conv_maps / (2 ** 0),(3,3),padding = 'same', activation='relu')(peak)

do4 = decoders(eo4, peak, max_conv_maps / (2 ** 1), (3,3), (3,3), 'same', 128, (3,3))
do3 = decoders(eo3, do4, max_conv_maps / (2 ** 2), (2,2), (2,2), 'same', 64, (3,3))
do2 = decoders(eo2, do3, max_conv_maps / (2 ** 3), (2,2), (2,2), 'same', 32, (3,3))
do1 = decoders(eo1, do2, max_conv_maps / (2 ** 4), (2,2), (2,2), 'same', 16, (3,3))

fin_out = Conv2D(1, (1,1), padding = 'same', activation='sigmoid')(do1)

model = Model(inputs = cnn_input, outputs = fin_out)
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

In [ ]:
checkpoint_callbk = tf.keras.callbacks.ModelCheckpoint(
    "best_model", # name of file to save the best model to
    monitor="val_accuracy", # prefix val to specify that we want the model with best macroF1 on the validation data
    verbose=1, # prints out when the model achieve a better epoch
    mode="max", # the monitored metric should be maximized
    save_freq="epoch", # clear
    save_best_only=True, # of course, if not, every time a new best is achieved will be savedf differently
    save_weights_only=True # this means that we don't have to save the architecture, if you change the architecture, you'll loose the old weights
)
kf = KFold(n_splits = 7, shuffle = True, random_state = 24)

for trainind, testind in kf.split(X_train,y_train):
    model.fit(x = X_train[trainind,:,:,:], y = y_train[trainind], validation_data = (X_train[testind,:,:,:], y_train[testind]), callbacks=[checkpoint_callbk], epochs=10,
    batch_size=5, verbose=0)
model.load_weights("best_model")

In [ ]:
model.evaluate(X_test, y_test, verbose = 1)